In [14]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten,MaxPool2D,BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import argparse, os
import pandas as pd
import numpy as np
import keras

In [2]:
from zipfile import ZipFile
file_name='/content/drive/MyDrive/freelance_projects/books-cover-prediction/dataset.zip'
with ZipFile (file_name,'r')as zip:
  zip.extractall()
  print('done')

done


In [3]:
training_dir = '/content/dataset'

In [4]:
train = pd.read_csv(os.path.join(training_dir, '/content/dataset/main.csv'))
# test = pd.read_csv(os.path.join(training_dir,"test" ,'test-classes.csv'))
# train_image = os.path.join(training_dir,"train" )
# test_image = os.path.join(training_dir, "test")

In [5]:
train

,image,name,author,format,book_depository_stars,price,currency,old_price,isbn,category,img_paths
0,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,The Barefoot Investor,Scott Pape,Paperback,4.5,14.06,$,17.60,9.780000e+12,Business-Finance-Law,dataset/Business-Finance-Law/0000001.jpg
1,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,How to Win Friends and Influence People,Dale Carnegie,Paperback,4.0,8.3,$,9.50,9.780000e+12,Business-Finance-Law,dataset/Business-Finance-Law/0000002.jpg
2,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,"Thinking, Fast and Slow",Daniel Kahneman,Paperback,4.0,11.5,$,15.00,9.780000e+12,Business-Finance-Law,dataset/Business-Finance-Law/0000003.jpg
3,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,Start With Why,Simon Sinek,Paperback,4.0,8.9,$,12.40,9.780000e+12,Business-Finance-Law,dataset/Business-Finance-Law/0000004.jpg
4,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,Man's Search For Meaning,Viktor E. Frankl,Paperback,4.5,9.66,$,NaN,9.780000e+12,Business-Finance-Law,dataset/Business-Finance-Law/0000005.jpg
...,...,...,...,...,...,...,...,...,...,...,...
1970,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,Who Was Albert Einstein?,Jess Brallier,Paperback,4.0,4.64,$,6.20,9.780000e+12,Childrens-Books,dataset/Childrens-Books/0000982.jpg
1971,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,A Day at the Fire Station,Garth Williams,Paperback,4.0,8.01,$,NaN,9.780000e+12,Childrens-Books,dataset/Childrens-Books/0000983.jpg
1972,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,We Are in a Book!,Mo Willems,Hardback,4.5,9.67,$,NaN,9.780000e+12,Childrens-Books,dataset/Childrens-Books/0000984.jpg
1973,https://d1w7fb2mkkr3kw.cloudfront.net/assets/i...,What to Do When You Dread Your Bed,Dawn Huebner,Paperback,4.5,15.27,$,16.19,9.780000e+12,Childrens-Books,dataset/Childrens-Books/0000985.jpg


In [6]:
print("Train Data Shape",train.shape)
# print("Test Data Shape",test.shape)
print("Column Name",train.columns)
# print("s3 image path",train_image)

Train Data Shape (1975, 11)
Column Name Index(['image', 'name', 'author', 'format', 'book_depository_stars', 'price',
       'currency', 'old_price', 'isbn', 'category', 'img_paths'],
      dtype='object')


In [7]:
#prepare train images
X = []
for i in tqdm(range(train.shape[0])):
  img = tf.keras.utils.load_img("/content/" + train['img_paths'][i],target_size=(400,400,3))
  img = tf.keras.utils.img_to_array(img)
  img = img/255
  X.append(img)
X= np.array(X)
print(X.shape)

100%|██████████| 1975/1975 [00:05<00:00, 362.31it/s]


(1975, 400, 400, 3)


In [8]:
#prepare train labels
y = train.drop(['image', 'name', 'author', 'format', 'book_depository_stars', 'price', 'currency', 'old_price', 'isbn', 'img_paths'],axis=1)
y = y.to_numpy()
print(y.shape)
print(X.shape)

(1975, 1)
(1975, 400, 400, 3)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

# seperate train test images and labels.
print(X_train.shape)
print(y_train.shape)
    
print(X_test.shape)
print(y_test.shape)

(1777, 400, 400, 3)
(1777, 1)
(198, 400, 400, 3)
(198, 1)


In [11]:
# #apply FIne tune methord for resnet50 model
# input_shape = (400 , 400 , 3)
# base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape=input_shape, include_top=False)

# for layer in base_model.layers:
#     layer.trainable = False
#     base_model.summary()
        
# for layer in base_model.layers[:]:
#     layer.trainable = False
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name, layer.trainable)
        
# # Make last block of the conv_base trainable:

# for layer in base_model.layers[:177]:
#     layer.trainable = False
# for layer in base_model.layers[177:]:
#     layer.trainable = True
    
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name, layer.trainable)

In [16]:
model = Sequential()
model.add(Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))


model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2,activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 398, 398, 16)      448       
                                                                 
 batch_normalization_4 (Batc  (None, 398, 398, 16)     64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 199, 199, 16)     0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 199, 199, 16)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 197, 197, 32)      4640      
                                                                 
 batch_normalization_5 (Batc  (None, 197, 197, 32)    

In [17]:
batch_size = 16
epochs = 1

In [ ]:
#compile your model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
#start training
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), batch_size=batch_size)
    
# save model
# model.save(f'{args.model_dir}/1')